# Advanced Compression Benchmarking Matrix

## Overview

This notebook extends the basic compression benchmarks with:
- 🔬 **Additional codecs**: Blosc-Zlib, GZip levels
- 📊 **Comprehensive matrices**: Heat maps comparing all configurations
- 🎯 **Multi-dimensional analysis**: Level × Codec × Chunk size
- 📈 **Trade-off analysis**: Compression vs Speed vs Quality
- 🗺️ **Interactive visualizations**: Color-coded performance matrices

## Compression Methods Tested

**Lossless Compression:**
- Blosc-Zstd (shuffle, bitshuffle, noshuffle)
- Blosc-LZ4 (shuffle, bitshuffle, noshuffle)
- Blosc-Zlib (shuffle, bitshuffle, noshuffle)
- Zstd (standalone)
- GZip (multiple levels)
- No compression (baseline)

**Configuration Space:**
- Compression levels: 1, 3, 5, 7, 9
- Chunk sizes: 64³, 128³, 256³
- Shuffle options: shuffle, bitshuffle, noshuffle

**Total combinations: ~150 configurations**

⚠️ **Note**: This is a comprehensive benchmark that will take ~30-60 minutes to complete.

In [ ]:
# Imports
import numpy as np
import pathlib
import time
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from cryoet_data_portal import Client, Dataset
import s3fs
import zarr
from zarr_benchmarks.read_write_zarr import read_write_zarr
from zarr_benchmarks import utils
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import mean_squared_error as mse
import warnings
warnings.filterwarnings('ignore')

# Set style for better visualizations
sns.set_style('whitegrid')
plt.rcParams['figure.dpi'] = 100

print("✓ All imports successful")

## Configuration

Adjust these settings to control the benchmark scope:

In [ ]:
# Data size
DOWNLOAD_SIZE = 256  # Cube size to download

# Benchmark scope
FULL_MATRIX = False  # Set True for complete matrix (takes longer)

if FULL_MATRIX:
    # Full comprehensive matrix
    CODECS = ['blosc_zstd', 'blosc_lz4', 'blosc_zlib', 'zstd', 'gzip', 'no_compression']
    SHUFFLES = ['shuffle', 'bitshuffle', 'noshuffle']
    LEVELS = [1, 3, 5, 7, 9]
    CHUNKS = [64, 128, 256]
else:
    # Focused matrix (faster, still comprehensive)
    CODECS = ['blosc_zstd', 'blosc_lz4', 'blosc_zlib', 'zstd', 'gzip', 'no_compression']
    SHUFFLES = ['shuffle', 'noshuffle']
    LEVELS = [1, 3, 5, 9]
    CHUNKS = [64, 128, 256]

ZARR_SPEC = 2

# Calculate total tests
blosc_tests = 3 * len(SHUFFLES) * len(LEVELS) * len(CHUNKS)  # 3 blosc variants
other_tests = 2 * len(LEVELS) * len(CHUNKS)  # zstd, gzip
baseline_tests = len(CHUNKS)  # no_compression
total_tests = blosc_tests + other_tests + baseline_tests

print(f"Configuration:")
print(f"  Download size: {DOWNLOAD_SIZE}³ = {(DOWNLOAD_SIZE**3 * 4)/(1024**2):.1f} MB")
print(f"  Mode: {'Full Matrix' if FULL_MATRIX else 'Focused Matrix'}")
print(f"  Codecs: {len(CODECS)}")
print(f"  Shuffles: {len(SHUFFLES)} (for Blosc)")
print(f"  Levels: {len(LEVELS)}")
print(f"  Chunk sizes: {len(CHUNKS)}")
print(f"  Total tests: ~{total_tests}")
print(f"  Estimated time: ~{total_tests * 0.5:.0f}-{total_tests * 1:.0f} minutes")

## 1. Download CryoET Data

In [ ]:
print("Connecting to CryoET Data Portal...")
client = Client()
dataset = Dataset.get_by_id(client, 10445)

print(f"✓ Dataset: {dataset.title}")

# Find tomogram
runs = list(dataset.runs)
selected_tomo = None
for run in runs:
    for tomo in list(run.tomograms):
        if (tomo.size_x >= DOWNLOAD_SIZE and 
            tomo.size_y >= DOWNLOAD_SIZE and 
            tomo.size_z >= DOWNLOAD_SIZE):
            selected_tomo = tomo
            break
    if selected_tomo:
        break

print(f"✓ Selected: {selected_tomo.name}")
print(f"  Size: {selected_tomo.size_x} × {selected_tomo.size_y} × {selected_tomo.size_z}")

# Download data
s3 = s3fs.S3FileSystem(anon=True)
zarr_path = selected_tomo.s3_omezarr_dir.replace('s3://', '')
store = s3fs.S3Map(root=zarr_path, s3=s3, check=False)
zarr_group = zarr.open(store, mode='r')
zarr_array = zarr_group['0']

# Download centered cube
actual_size = min(DOWNLOAD_SIZE, min(zarr_array.shape))
z_c = zarr_array.shape[0] // 2
y_c = zarr_array.shape[1] // 2
x_c = zarr_array.shape[2] // 2

z_start = max(0, z_c - actual_size // 2)
z_end = min(zarr_array.shape[0], z_start + actual_size)
y_start = max(0, y_c - actual_size // 2)
y_end = min(zarr_array.shape[1], y_start + actual_size)
x_start = max(0, x_c - actual_size // 2)
x_end = min(zarr_array.shape[2], x_start + actual_size)

print(f"\nDownloading {actual_size}³ cube...")
start_time = time.time()
reference_data = np.array(zarr_array[z_start:z_end, y_start:y_end, x_start:x_end])
download_time = time.time() - start_time

print(f"✓ Downloaded in {download_time:.2f}s")
print(f"  Shape: {reference_data.shape}")
print(f"  Size: {reference_data.nbytes / (1024**2):.2f} MB")

## 2. Run Comprehensive Benchmark Matrix

In [ ]:
# Setup
output_dir = pathlib.Path("data/output/compression_matrix")
output_dir.mkdir(parents=True, exist_ok=True)

all_results = []

def calculate_metrics(original, compressed):
    """Calculate image quality metrics"""
    try:
        orig_norm = (original - original.min()) / (original.max() - original.min() + 1e-10)
        comp_norm = (compressed - compressed.min()) / (compressed.max() - compressed.min() + 1e-10)
        mid = original.shape[0] // 2
        ssim_val = ssim(orig_norm[mid], comp_norm[mid], data_range=1.0)
        data_range = original.max() - original.min()
        psnr_val = psnr(original, compressed, data_range=data_range)
        mse_val = mse(original, compressed)
        return ssim_val, psnr_val, mse_val
    except:
        return None, None, None

print("✓ Setup complete")

In [ ]:
# Run comprehensive benchmark
import sys

test_num = 0
start_time = time.time()

for codec in CODECS:
    print(f"\n{'='*70}")
    print(f"Testing: {codec.upper()}")
    print(f"{'='*70}")
    
    if codec == 'no_compression':
        for chunk_size in CHUNKS:
            test_num += 1
            print(f"[{test_num}/{total_tests}] chunk={chunk_size}...", end=' ')
            sys.stdout.flush()
            
            store_path = output_dir / f"{codec}_c{chunk_size}.zarr"
            utils.remove_output_dir(store_path)
            chunks = (chunk_size, chunk_size, chunk_size)
            
            t0 = time.time()
            read_write_zarr.write_zarr_array(
                reference_data, store_path, overwrite=False,
                chunks=chunks, compressor=None, zarr_spec=ZARR_SPEC
            )
            write_time = time.time() - t0
            
            t0 = time.time()
            read_back = read_write_zarr.read_zarr_array(store_path)
            read_time = time.time() - t0
            
            ratio = 1.0
            size_mb = utils.get_directory_size(store_path) / (1024**2)
            ssim_val, psnr_val, mse_val = calculate_metrics(reference_data, read_back)
            
            all_results.append({
                'codec': codec,
                'shuffle': 'N/A',
                'level': 0,
                'chunk_size': chunk_size,
                'write_time': write_time,
                'read_time': read_time,
                'ratio': ratio,
                'size_mb': size_mb,
                'ssim': ssim_val,
                'psnr': psnr_val,
                'mse': mse_val,
                'throughput_write_mbs': (reference_data.nbytes / (1024**2)) / write_time,
                'throughput_read_mbs': (reference_data.nbytes / (1024**2)) / read_time
            })
            
            print(f"W:{write_time:.3f}s R:{read_time:.3f}s")
    
    elif 'blosc' in codec:
        cname = codec.split('_')[1]
        
        for shuffle in SHUFFLES:
            for level in LEVELS:
                for chunk_size in CHUNKS:
                    test_num += 1
                    print(f"[{test_num}/{total_tests}] {shuffle[:3]}, L{level}, c{chunk_size}...", end=' ')
                    sys.stdout.flush()
                    
                    store_path = output_dir / f"{codec}_{shuffle}_l{level}_c{chunk_size}.zarr"
                    utils.remove_output_dir(store_path)
                    chunks = (chunk_size, chunk_size, chunk_size)
                    compressor = read_write_zarr.get_blosc_compressor(cname, level, shuffle)
                    
                    t0 = time.time()
                    read_write_zarr.write_zarr_array(
                        reference_data, store_path, overwrite=False,
                        chunks=chunks, compressor=compressor, zarr_spec=ZARR_SPEC
                    )
                    write_time = time.time() - t0
                    
                    t0 = time.time()
                    read_back = read_write_zarr.read_zarr_array(store_path)
                    read_time = time.time() - t0
                    
                    ratio = read_write_zarr.get_compression_ratio(store_path)
                    size_mb = utils.get_directory_size(store_path) / (1024**2)
                    ssim_val, psnr_val, mse_val = calculate_metrics(reference_data, read_back)
                    
                    all_results.append({
                        'codec': codec,
                        'shuffle': shuffle,
                        'level': level,
                        'chunk_size': chunk_size,
                        'write_time': write_time,
                        'read_time': read_time,
                        'ratio': ratio,
                        'size_mb': size_mb,
                        'ssim': ssim_val,
                        'psnr': psnr_val,
                        'mse': mse_val,
                        'throughput_write_mbs': (reference_data.nbytes / (1024**2)) / write_time,
                        'throughput_read_mbs': (reference_data.nbytes / (1024**2)) / read_time
                    })
                    
                    print(f"W:{write_time:.3f}s R:{read_time:.3f}s {ratio:.2f}x")
    
    else:
        for level in LEVELS:
            for chunk_size in CHUNKS:
                test_num += 1
                print(f"[{test_num}/{total_tests}] L{level}, c{chunk_size}...", end=' ')
                sys.stdout.flush()
                
                store_path = output_dir / f"{codec}_l{level}_c{chunk_size}.zarr"
                utils.remove_output_dir(store_path)
                chunks = (chunk_size, chunk_size, chunk_size)
                
                if codec == 'zstd':
                    compressor = read_write_zarr.get_zstd_compressor(level)
                else:
                    compressor = read_write_zarr.get_gzip_compressor(level)
                
                t0 = time.time()
                read_write_zarr.write_zarr_array(
                    reference_data, store_path, overwrite=False,
                    chunks=chunks, compressor=compressor, zarr_spec=ZARR_SPEC
                )
                write_time = time.time() - t0
                
                t0 = time.time()
                read_back = read_write_zarr.read_zarr_array(store_path)
                read_time = time.time() - t0
                
                ratio = read_write_zarr.get_compression_ratio(store_path)
                size_mb = utils.get_directory_size(store_path) / (1024**2)
                ssim_val, psnr_val, mse_val = calculate_metrics(reference_data, read_back)
                
                all_results.append({
                    'codec': codec,
                    'shuffle': 'N/A',
                    'level': level,
                    'chunk_size': chunk_size,
                    'write_time': write_time,
                    'read_time': read_time,
                    'ratio': ratio,
                    'size_mb': size_mb,
                    'ssim': ssim_val,
                    'psnr': psnr_val,
                    'mse': mse_val,
                    'throughput_write_mbs': (reference_data.nbytes / (1024**2)) / write_time,
                    'throughput_read_mbs': (reference_data.nbytes / (1024**2)) / read_time
                })
                
                print(f"W:{write_time:.3f}s R:{read_time:.3f}s {ratio:.2f}x")

total_time = time.time() - start_time
print(f"\n✓ Completed {len(all_results)} tests in {total_time/60:.1f} minutes")

## 3. Create Results DataFrame

In [ ]:
# Create DataFrame
df = pd.DataFrame(all_results)

# Add derived metrics
df['total_time'] = df['write_time'] + df['read_time']
df['efficiency_score'] = df['ratio'] / df['total_time']  # Compression per second
df['codec_config'] = df.apply(
    lambda x: f"{x['codec']}_{x['shuffle']}_{x['level']}" if x['shuffle'] != 'N/A' else f"{x['codec']}_L{x['level']}",
    axis=1
)

# Save to CSV
csv_path = output_dir / f"compression_matrix_{DOWNLOAD_SIZE}cube.csv"
df.to_csv(csv_path, index=False)
print(f"✓ Saved to: {csv_path}")

# Display summary
print(f"\nDataset summary:")
print(f"  Total configurations: {len(df)}")
print(f"  Codecs tested: {df['codec'].nunique()}")
print(f"  Compression levels: {sorted(df['level'].unique())}")
print(f"  Chunk sizes: {sorted(df['chunk_size'].unique())}")

df.head(10)

## 4. Compression Ratio Matrix (Heat Map)

In [ ]:
# Create compression ratio matrix for each chunk size
fig, axes = plt.subplots(1, len(CHUNKS), figsize=(6*len(CHUNKS), 6))
if len(CHUNKS) == 1:
    axes = [axes]

fig.suptitle('Compression Ratio Matrix by Chunk Size', fontsize=16, fontweight='bold', y=1.02)

for idx, chunk_size in enumerate(CHUNKS):
    # Filter data for this chunk size
    chunk_df = df[df['chunk_size'] == chunk_size].copy()
    
    # Create pivot table: codec vs level
    # For blosc codecs with shuffle, use shuffle variant as codec name
    chunk_df['display_codec'] = chunk_df.apply(
        lambda x: f"{x['codec']}_{x['shuffle'][:3]}" if 'blosc' in x['codec'] and x['shuffle'] != 'N/A' else x['codec'],
        axis=1
    )
    
    pivot = chunk_df.pivot_table(
        values='ratio',
        index='display_codec',
        columns='level',
        aggfunc='mean'
    )
    
    # Create heatmap
    sns.heatmap(
        pivot,
        annot=True,
        fmt='.2f',
        cmap='YlGnBu',
        cbar_kws={'label': 'Compression Ratio'},
        ax=axes[idx],
        vmin=1.0,
        vmax=df['ratio'].max()
    )
    
    axes[idx].set_title(f'Chunk Size: {chunk_size}³', fontsize=12, fontweight='bold')
    axes[idx].set_xlabel('Compression Level', fontsize=10)
    axes[idx].set_ylabel('Codec', fontsize=10)

plt.tight_layout()
plt.savefig(output_dir / 'compression_ratio_matrix.png', dpi=150, bbox_inches='tight')
plt.show()

print("✓ Compression ratio matrix created")

## 5. Read/Write Speed Matrices

In [ ]:
# Create speed matrices
fig, axes = plt.subplots(2, len(CHUNKS), figsize=(6*len(CHUNKS), 12))
if len(CHUNKS) == 1:
    axes = axes.reshape(-1, 1)

fig.suptitle('Read/Write Throughput (MB/s) by Chunk Size', fontsize=16, fontweight='bold', y=1.00)

for idx, chunk_size in enumerate(CHUNKS):
    chunk_df = df[df['chunk_size'] == chunk_size].copy()
    chunk_df['display_codec'] = chunk_df.apply(
        lambda x: f"{x['codec']}_{x['shuffle'][:3]}" if 'blosc' in x['codec'] and x['shuffle'] != 'N/A' else x['codec'],
        axis=1
    )
    
    # Write throughput
    pivot_write = chunk_df.pivot_table(
        values='throughput_write_mbs',
        index='display_codec',
        columns='level',
        aggfunc='mean'
    )
    
    sns.heatmap(
        pivot_write,
        annot=True,
        fmt='.1f',
        cmap='RdYlGn',
        cbar_kws={'label': 'Write Throughput (MB/s)'},
        ax=axes[0, idx]
    )
    
    axes[0, idx].set_title(f'Write: Chunk {chunk_size}³', fontsize=11, fontweight='bold')
    axes[0, idx].set_xlabel('Compression Level', fontsize=9)
    axes[0, idx].set_ylabel('Codec', fontsize=9)
    
    # Read throughput
    pivot_read = chunk_df.pivot_table(
        values='throughput_read_mbs',
        index='display_codec',
        columns='level',
        aggfunc='mean'
    )
    
    sns.heatmap(
        pivot_read,
        annot=True,
        fmt='.1f',
        cmap='RdYlGn',
        cbar_kws={'label': 'Read Throughput (MB/s)'},
        ax=axes[1, idx]
    )
    
    axes[1, idx].set_title(f'Read: Chunk {chunk_size}³', fontsize=11, fontweight='bold')
    axes[1, idx].set_xlabel('Compression Level', fontsize=9)
    axes[1, idx].set_ylabel('Codec', fontsize=9)

plt.tight_layout()
plt.savefig(output_dir / 'speed_matrices.png', dpi=150, bbox_inches='tight')
plt.show()

print("✓ Speed matrices created")

## 6. Compression vs Speed Trade-off Analysis

In [ ]:
# Scatter plot: Compression ratio vs Total time, colored by codec
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Plot 1: Compression vs Write time
for codec in df['codec'].unique():
    codec_df = df[df['codec'] == codec]
    axes[0].scatter(
        codec_df['write_time'],
        codec_df['ratio'],
        label=codec,
        alpha=0.6,
        s=100
    )

axes[0].set_xlabel('Write Time (s)', fontsize=12)
axes[0].set_ylabel('Compression Ratio', fontsize=12)
axes[0].set_title('Compression Ratio vs Write Speed', fontsize=14, fontweight='bold')
axes[0].legend(bbox_to_anchor=(1.05, 1), loc='upper left')
axes[0].grid(True, alpha=0.3)

# Plot 2: Compression vs Read time
for codec in df['codec'].unique():
    codec_df = df[df['codec'] == codec]
    axes[1].scatter(
        codec_df['read_time'],
        codec_df['ratio'],
        label=codec,
        alpha=0.6,
        s=100
    )

axes[1].set_xlabel('Read Time (s)', fontsize=12)
axes[1].set_ylabel('Compression Ratio', fontsize=12)
axes[1].set_title('Compression Ratio vs Read Speed', fontsize=14, fontweight='bold')
axes[1].legend(bbox_to_anchor=(1.05, 1), loc='upper left')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(output_dir / 'compression_speed_tradeoff.png', dpi=150, bbox_inches='tight')
plt.show()

print("✓ Trade-off analysis complete")

## 7. Pareto Frontier Analysis

Find optimal configurations on the Pareto frontier (best compression for given speed)

In [ ]:
# Find Pareto frontier for compression vs total time
def is_pareto_efficient(costs):
    """
    Find Pareto efficient points
    costs: (n_points, n_costs) array
    """
    is_efficient = np.ones(costs.shape[0], dtype=bool)
    for i, c in enumerate(costs):
        if is_efficient[i]:
            # Keep points that are not dominated
            is_efficient[is_efficient] = np.any(costs[is_efficient] < c, axis=1)
            is_efficient[i] = True
    return is_efficient

# Prepare data for Pareto analysis
# We want to minimize time and maximize ratio
# So we use (time, -ratio) as costs
costs = np.column_stack([
    df['total_time'].values,
    -df['ratio'].values  # Negative because we want to maximize
])

pareto_mask = is_pareto_efficient(costs)
pareto_df = df[pareto_mask].copy()
pareto_df = pareto_df.sort_values('total_time')

# Visualize Pareto frontier
fig, ax = plt.subplots(figsize=(12, 8))

# Plot all points
ax.scatter(
    df['total_time'],
    df['ratio'],
    alpha=0.3,
    s=50,
    label='All configurations',
    color='gray'
)

# Plot Pareto frontier
ax.scatter(
    pareto_df['total_time'],
    pareto_df['ratio'],
    alpha=0.8,
    s=200,
    label='Pareto optimal',
    color='red',
    edgecolors='black',
    linewidths=2
)

# Connect Pareto points
ax.plot(
    pareto_df['total_time'],
    pareto_df['ratio'],
    'r--',
    alpha=0.5,
    linewidth=2
)

# Annotate Pareto points
for _, row in pareto_df.iterrows():
    label = f"{row['codec']}\nL{row['level']}, c{row['chunk_size']}"
    ax.annotate(
        label,
        (row['total_time'], row['ratio']),
        xytext=(10, 10),
        textcoords='offset points',
        fontsize=8,
        bbox=dict(boxstyle='round,pad=0.5', facecolor='yellow', alpha=0.7),
        arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0')
    )

ax.set_xlabel('Total Time (Write + Read) [s]', fontsize=12)
ax.set_ylabel('Compression Ratio', fontsize=12)
ax.set_title('Pareto Frontier: Optimal Compression vs Speed Trade-offs', fontsize=14, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(output_dir / 'pareto_frontier.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"✓ Found {len(pareto_df)} Pareto optimal configurations")
print("\nPareto Optimal Configurations:")
print(pareto_df[['codec', 'shuffle', 'level', 'chunk_size', 'total_time', 'ratio', 'size_mb']].to_string(index=False))

## 8. Comprehensive Summary & Recommendations

In [ ]:
print("="*80)
print("COMPREHENSIVE COMPRESSION MATRIX SUMMARY")
print("="*80)

print(f"\nDataset: {DOWNLOAD_SIZE}³ CryoET tomogram ({(DOWNLOAD_SIZE**3 * 4)/(1024**2):.1f} MB)")
print(f"Configurations tested: {len(df)}")
print(f"Total benchmark time: {total_time/60:.1f} minutes")

print("\n" + "="*80)
print("1. BEST OVERALL PERFORMERS")
print("="*80)

# Best compression
best_comp = df.loc[df['ratio'].idxmax()]
print(f"\n🏆 Best Compression Ratio: {best_comp['ratio']:.3f}×")
print(f"   Codec: {best_comp['codec']} (shuffle={best_comp['shuffle']}, level={best_comp['level']}, chunk={best_comp['chunk_size']})")
print(f"   Size: {best_comp['size_mb']:.2f} MB (saves {64 - best_comp['size_mb']:.2f} MB)")
print(f"   Time: Write={best_comp['write_time']:.3f}s, Read={best_comp['read_time']:.3f}s")

# Fastest write
best_write = df.loc[df['write_time'].idxmin()]
print(f"\n⚡ Fastest Write: {best_write['write_time']:.3f}s ({best_write['throughput_write_mbs']:.1f} MB/s)")
print(f"   Codec: {best_write['codec']} (level={best_write['level']}, chunk={best_write['chunk_size']})")
print(f"   Compression: {best_write['ratio']:.3f}×")

# Fastest read
best_read = df.loc[df['read_time'].idxmin()]
print(f"\n⚡ Fastest Read: {best_read['read_time']:.3f}s ({best_read['throughput_read_mbs']:.1f} MB/s)")
print(f"   Codec: {best_read['codec']} (level={best_read['level']}, chunk={best_read['chunk_size']})")
print(f"   Compression: {best_read['ratio']:.3f}×")

# Best efficiency
best_eff = df.loc[df['efficiency_score'].idxmax()]
print(f"\n⚙️  Best Efficiency (compression/time): {best_eff['efficiency_score']:.3f}")
print(f"   Codec: {best_eff['codec']} (shuffle={best_eff['shuffle']}, level={best_eff['level']}, chunk={best_eff['chunk_size']})")
print(f"   Ratio: {best_eff['ratio']:.3f}×, Time: {best_eff['total_time']:.3f}s")

print("\n" + "="*80)
print("2. CODEC COMPARISON (Chunk=128, Level=5)")
print("="*80)

comparison_df = df[(df['chunk_size']==128) & (df['level'].isin([5, 0]))].copy()
comparison_df = comparison_df.sort_values('ratio', ascending=False)

print(f"\n{'Codec':<25} {'Shuffle':<12} {'Ratio':<8} {'Write(s)':<10} {'Read(s)':<10} {'Size(MB)':<10}")
print("-"*85)

for _, row in comparison_df.head(10).iterrows():
    codec_name = f"{row['codec']}"
    shuffle_str = row['shuffle'] if row['shuffle'] != 'N/A' else '-'
    print(f"{codec_name:<25} {shuffle_str:<12} {row['ratio']:<8.3f} {row['write_time']:<10.3f} {row['read_time']:<10.3f} {row['size_mb']:<10.2f}")

print("\n" + "="*80)
print("3. CHUNK SIZE IMPACT")
print("="*80)

for chunk in CHUNKS:
    chunk_subset = df[df['chunk_size'] == chunk]
    print(f"\nChunk size {chunk}³:")
    print(f"  Avg compression ratio: {chunk_subset['ratio'].mean():.3f}×")
    print(f"  Avg write time: {chunk_subset['write_time'].mean():.3f}s")
    print(f"  Avg read time: {chunk_subset['read_time'].mean():.3f}s")
    print(f"  Best ratio: {chunk_subset['ratio'].max():.3f}× ({chunk_subset.loc[chunk_subset['ratio'].idxmax(), 'codec']})")

print("\n" + "="*80)
print("4. RECOMMENDATIONS BY USE CASE")
print("="*80)

print("\n📦 For Long-term Archival (Best Compression):")
print(f"   → {best_comp['codec']} with {best_comp['shuffle']} shuffle, level {best_comp['level']}, chunk {best_comp['chunk_size']}³")
print(f"   → Saves {((1 - best_comp['size_mb']/64) * 100):.1f}% storage space")

print("\n⚡ For Real-time Processing (Best Speed):")
fastest_overall = df.loc[df['total_time'].idxmin()]
print(f"   → {fastest_overall['codec']} level {fastest_overall['level']}, chunk {fastest_overall['chunk_size']}³")
print(f"   → Total time: {fastest_overall['total_time']:.3f}s (write + read)")

print("\n⚙️  For General Use (Best Balance):")
# Find best balance: good compression (>1.1x) and fast (<0.1s total)
balanced = df[(df['ratio'] > 1.1) & (df['total_time'] < 0.1)]
if not balanced.empty:
    best_balanced = balanced.loc[balanced['efficiency_score'].idxmax()]
    print(f"   → {best_balanced['codec']} level {best_balanced['level']}, chunk {best_balanced['chunk_size']}³")
    print(f"   → Ratio: {best_balanced['ratio']:.3f}×, Time: {best_balanced['total_time']:.3f}s")
else:
    print(f"   → {best_eff['codec']} level {best_eff['level']}, chunk {best_eff['chunk_size']}³")
    print(f"   → Efficiency score: {best_eff['efficiency_score']:.3f}")

print("\n" + "="*80)
print("✅ COMPREHENSIVE MATRIX ANALYSIS COMPLETE")
print("="*80)

print(f"\nResults saved to: {output_dir}")
print("Files generated:")
print(f"  - compression_matrix_{DOWNLOAD_SIZE}cube.csv")
print("  - compression_ratio_matrix.png")
print("  - speed_matrices.png")
print("  - compression_speed_tradeoff.png")
print("  - pareto_frontier.png")

## 9. Export Summary Report

In [ ]:
# Create markdown summary report
report = f"""# Compression Benchmarking Matrix Report

**Dataset:** CryoET Tomogram {DOWNLOAD_SIZE}³ ({(DOWNLOAD_SIZE**3 * 4)/(1024**2):.1f} MB)
**Date:** {time.strftime('%Y-%m-%d %H:%M:%S')}
**Configurations Tested:** {len(df)}
**Benchmark Time:** {total_time/60:.1f} minutes

## Best Performers

### 🏆 Best Compression
- **Codec:** {best_comp['codec']} (shuffle={best_comp['shuffle']}, level={best_comp['level']}, chunk={best_comp['chunk_size']}³)
- **Ratio:** {best_comp['ratio']:.3f}×
- **Size:** {best_comp['size_mb']:.2f} MB (saves {64 - best_comp['size_mb']:.2f} MB, {((1 - best_comp['size_mb']/64) * 100):.1f}%)
- **Time:** Write={best_comp['write_time']:.3f}s, Read={best_comp['read_time']:.3f}s

### ⚡ Fastest Write
- **Codec:** {best_write['codec']} (level={best_write['level']}, chunk={best_write['chunk_size']}³)
- **Time:** {best_write['write_time']:.3f}s ({best_write['throughput_write_mbs']:.1f} MB/s)
- **Ratio:** {best_write['ratio']:.3f}×

### ⚡ Fastest Read
- **Codec:** {best_read['codec']} (level={best_read['level']}, chunk={best_read['chunk_size']}³)
- **Time:** {best_read['read_time']:.3f}s ({best_read['throughput_read_mbs']:.1f} MB/s)
- **Ratio:** {best_read['ratio']:.3f}×

## Pareto Optimal Configurations

{pareto_df[['codec', 'shuffle', 'level', 'chunk_size', 'ratio', 'total_time']].to_markdown(index=False)}

## Recommendations

**For Archival Storage:**
- Use: {best_comp['codec']} with {best_comp['shuffle']} shuffle, level {best_comp['level']}, chunk {best_comp['chunk_size']}³
- Benefit: {((1 - best_comp['size_mb']/64) * 100):.1f}% storage savings

**For Real-time Processing:**
- Use: {fastest_overall['codec']} level {fastest_overall['level']}, chunk {fastest_overall['chunk_size']}³
- Benefit: {fastest_overall['total_time']:.3f}s total latency

**For General Use:**
- Use: {best_eff['codec']} level {best_eff['level']}, chunk {best_eff['chunk_size']}³
- Benefit: Best efficiency score ({best_eff['efficiency_score']:.3f})
"""

report_path = output_dir / f"compression_matrix_report_{DOWNLOAD_SIZE}cube.md"
with open(report_path, 'w') as f:
    f.write(report)

print(f"✓ Summary report saved to: {report_path}")

## Next Steps

**To test different data:**
1. Change `DOWNLOAD_SIZE` to test on larger volumes
2. Change dataset ID to test on different tomograms

**To expand the matrix:**
1. Set `FULL_MATRIX = True` for complete testing
2. Add custom codec configurations

**Analysis ideas:**
- Compare results across multiple datasets
- Test with different data types (segmentations, etc.)
- Analyze impact of data characteristics on compression

**Resources:**
- CryoET Portal: https://cryoetdataportal.czscience.com/
- Zarr Docs: https://zarr.readthedocs.io/
- Numcodecs: https://numcodecs.readthedocs.io/